In [1]:
import os
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers, Model
import pathlib
from tensorflow.keras import Input
import numpy as np
from tensorflow.keras.layers import Lambda, Conv2D, BatchNormalization, MaxPooling2D, Conv2DTranspose, concatenate, Concatenate
from tensorflow.keras.metrics import MeanIoU

In [2]:
#Deviding data by batches

In [3]:
def process_path(image_path, mask_path):
    image = tf.io.read_file(image_path)
    mask = tf.io.read_file(mask_path)
    image = tf.image.decode_png(image, channels = 3)
    mask = tf.image.decode_png(mask, channels = 1)
    return image, mask

In [4]:
def batch_data (path_X, path_y, batch_size):
    X_names = os.listdir(path_X)
    X_path = [f'{path_X}/{file}' for file in X_names]
    y_names = os.listdir(path_y)
    y_path = [f'{path_y}/{file}' for file in y_names]
    ds_train = tf.data.Dataset.from_tensor_slices((X_path, y_path))
    return ds_train.map(process_path).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [9]:
def unet_model(img_height, img_width, channels):

    #Input
    inputs = Input((img_height, img_width, channels), name="input")
    normalisation = Lambda(lambda x: x / 255, name="normalisation")(inputs) #Normalize the pixels by dividing by 255

#     #Encoder where we are extracting the features
#     convolution0 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same', name="conv2D1tototootototo")(normalisation)
#     convolution0 = BatchNormalization(name="batchnorm1")(convolution0) #other option is to do dropout by batch is faster 
#     convolution0 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(convolution0)
#     pooling0 = MaxPooling2D((2, 2))(convolution0)
      
    convolution1 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(normalisation)
    convolution1 = BatchNormalization()(convolution1) 
    convolution1 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(convolution1)
    pooling1 = MaxPooling2D((2, 2))(convolution1)
    
    convolution2 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(pooling1)
    convolution2 = BatchNormalization()(convolution2)
    convolution2 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(convolution2)
    pooling2 = MaxPooling2D((2, 2))(convolution2)
     
    convolution3 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(pooling2)
    convolution3 = BatchNormalization()(convolution3)
    convolution3 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(convolution3)
    pooling3 = MaxPooling2D((2, 2))(convolution3)
     
    convolution4 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(pooling3)
    convolution4 = BatchNormalization()(convolution4)
    convolution4 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(convolution4)
    pooling4 = MaxPooling2D(pool_size=(2, 2))(convolution4)

    #Bottleneck at the base of the U-net 
    convolution5 = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(pooling4)
    convolution5 = BatchNormalization()(convolution5)
    convolution5 = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(convolution5)
    
    #Decoder where we are indicating to the model the precise location of the features 
    transconv6 = Conv2DTranspose(128, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution5)
    transconv6 = Concatenate()([transconv6, convolution4])
    convolution6 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(transconv6)
    convolution6 = BatchNormalization()(convolution6)
    convolution6 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(convolution6)
     
    transconv7 = Conv2DTranspose(64, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution6)
    transconv7 = Concatenate()([transconv7, convolution3])
    convolution7 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(transconv7)
    convolution7 = BatchNormalization()(convolution7)
    convolution7 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(convolution7)
     
    transconv8 = Conv2DTranspose(32, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution7)
    transconv8 = Concatenate()([transconv8, convolution2])
    convolution8 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(transconv8)
    convolution8 = BatchNormalization()(convolution8)
    convolution8 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(convolution8)
     
    transconv9 = Conv2DTranspose(16, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution8)
    transconv9 = Concatenate()([transconv9, convolution1])
    convolution9 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(transconv9)
    convolution9 = BatchNormalization()(convolution9)
    convolution9 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(convolution9)
  
   
    
    outputs = Conv2D(1, kernel_size=(1, 1), activation='sigmoid')(convolution9)
     
    model = Model(inputs=inputs, outputs=outputs)
    jaccard = MeanIoU(2)
    
    #loss options include: binary_crossentropy, IoU Loss (Jaccard Index), dice loss
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics="accuracy") 
    
#     model.summary()
    
    return model

In [15]:
channels=3
img_height = 256
img_width = 256

In [16]:
model = unet_model(img_height, img_width, channels)

In [17]:
path_X = '/home/zhlj/code/solar_panel_segmentation/tests/Small_dataset/Images'
path_y = '/home/zhlj/code/solar_panel_segmentation/tests/Small_dataset/Target'
batch_size = 8

In [18]:

tb_callback = tf.keras.callbacks.TensorBoard(log_dir = "logs" , histogram_freq=1)

In [ ]:
history = model.fit(batch_data(path_X,path_y,batch_size), epochs=100, callbacks=[tb_callback])

Epoch 1/100


2022-11-30 21:34:06.554661: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8700
2022-11-30 21:34:08.914673: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-30 21:34:11.370896: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.28GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-11-30 21:34:13.627589: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.15GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


276/276 [==============================] - 126s 404ms/step - loss: -10680.3926 - accuracy: 0.1935
Epoch 2/100
276/276 [==============================] - 106s 385ms/step - loss: -98568.5859 - accuracy: 0.0000e+00
Epoch 3/100
276/276 [==============================] - 104s 377ms/step - loss: -366432.5938 - accuracy: 0.0000e+00
Epoch 4/100
276/276 [==============================] - 107s 387ms/step - loss: -898590.5000 - accuracy: 0.0000e+00
Epoch 5/100
276/276 [==============================] - 107s 388ms/step - loss: -1773448.0000 - accuracy: 0.0000e+00
Epoch 6/100
276/276 [==============================] - 107s 387ms/step - loss: -3043322.2500 - accuracy: 0.0000e+00
Epoch 7/100
276/276 [==============================] - 106s 386ms/step - loss: -4751603.0000 - accuracy: 0.0000e+00
Epoch 8/100
276/276 [==============================] - 107s 388ms/step - loss: -6899042.5000 - accuracy: 0.0000e+00
Epoch 9/100
276/276 [==============================] - 104s 379ms/step - loss: -9558710.0000 -

In [ ]:
model.layers

In [ ]:
image

In [ ]:
np.array(image).astype(float)

In [ ]:
tf.cast(image, 'float32')

In [ ]:
tf.cast(image, 'float32')

In [ ]:
model.summary()

In [ ]:
output1 = model.layers[1]
output1(tf.cast(image, 'float32'))

In [ ]:
layer_names = [layer.name for layer in model.layers]
layer_names

In [ ]:
layer_outputs = [layer.output for layer in model.layers]
layer_outputs

In [ ]:
model.input

In [ ]:
feature_map_model = tf.keras.models.Model(inputs=model.input, outputs=layer_outputs[1:3])


In [ ]:
feature_map_model(tf.cast(image, 'float32'))

In [ ]:
model.layers[1].weights[0][:,:,:,0]

In [ ]:
model.layers[2].weights